# Creates a model archive file to be used in the torchserve for deployment

## Downloads the model from the s3 link 

In [1]:
# Download the model and rename it to pytorch_model.bin, then move it to model folder

In [19]:
!wget http://dl.fbaipublicfiles.com/dynabench/sentiment/roberta_round1.bin

--2020-06-30 10:21:10--  http://dl.fbaipublicfiles.com/dynabench/hs/roberta_round1.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 501003010 (478M) [application/octet-stream]
Saving to: ‘roberta_round1.bin’

roberta_round1.bin  100%[===================>] 477.79M  17.0MB/s    in 12s     

2020-06-30 10:21:23 (40.2 MB/s) - ‘roberta_round1.bin’ saved [501003010/501003010]



In [ ]:
## Installs torchserve and torch-model-archiver to be used in this kernel

In [6]:
!pip install torchserve torch-model-archiver

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 757kB 22.0MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 13.4MB/s ta 0:00:01
    100% |████████████████████████████████| 1.2MB 24.6MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 32.4MB/s ta 0:00:01
    100% |████████████████████████████████| 665kB 23.2MB/s ta 0:00:01
    100% |████████████████████████████████| 307kB 45.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for sacremoses ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 

### Create torchscript file 

In [12]:
#The model should be inside the model folder,go to the model folder and execute the below command
#python3 create_torchscript.py 
#then move the .pt file from transformer_model folder to model folder

Transformers version 3.0.0
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
Use model and tokenizer from directory roberta-base


## Model Archiving

In [3]:
# Creates session to get the defualt bucket name to push the tar file
import boto3, time, json, sagemaker
sess = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=sess)

model_file_name = "sent_class_r1"
bucket_name = "sagemaker-us-west-1-096166425824"
prefix = 'torchserve'


In [4]:
# !torch-model-archiver --help

usage: torch-model-archiver [-h] --model-name MODEL_NAME --serialized-file
                            SERIALIZED_FILE [--model-file MODEL_FILE]
                            --handler HANDLER [--source-vocab SOURCE_VOCAB]
                            [--extra-files EXTRA_FILES]
                            [--runtime {python,python2,python3}]
                            [--export-path EXPORT_PATH]
                            [--archive-format {tgz,no-archive,default}] [-f]
                            -v VERSION

Torch Model Archiver Tool

optional arguments:
  -h, --help            show this help message and exit
  --model-name MODEL_NAME
                        Exported model name. Exported file will be named as
                        model-name.mar and saved in current working directory if no --export-path is
                        specified, else it will be saved under the export path
  --serialized-file SERIALIZED_FILE
                        Path to .pt or .pth file containing stat

In [2]:
#Make sure you have the following files in the model folder pytorch_model.pt,Transformer_handler_generalized.py,vocab.json,setup_config.json,special_tokens_map.json,settings.py,tokenizer_config.json,merges.txt,qa_utils.py,config.json

In [16]:
!torch-model-archiver --model-name sent_class_r1 --version 1.0 --serialized-file ./model/pytorch_model.pt --handler ./model/Transformer_handler_generalized.py --extra-files "./model/vocab.json,./model/setup_config.json,./model/special_tokens_map.json,./model/settings.py,./model/tokenizer_config.json,./model/merges.txt"

In [ ]:
#The mar file will be present in the home directory
#This creates a tar file to be used in the sagemaker deployment

In [17]:
!tar cvfz {model_file_name}.tar.gz {model_file_name}.mar

sent_class_r1.mar


In [ ]:
#Moves the tar file to mars folder
!mv {model_file_name mars}.tar.gz {model_file_name}.mar ./mars/

In [18]:
# change the folder name before the last slash based on the task
!aws s3 cp mars/{model_file_name}.tar.gz s3://{bucket_name}/{prefix}/models/sent_class/
#The below s3 link will be given as the model data for sagemaker while deployment

upload: mars/sent_class_r1.tar.gz to s3://sagemaker-us-west-1-096166425824/torchserve/models/sent_class/sent_class_r1.tar.gz
